# Stock Comparison Tool

Trying to implement something similar to https://www.barchart.com/myBarchart/quotes/SPY/interactive-chart but in Gradio.

The code in the first code block of this notebook is executed when the `app.py` file is run. This notebook is used for the simplicity of interactive development and testing.

In [87]:
import yfinance as yf
import pandas as pd
import gradio as gr
from datetime import timedelta

# retrieve historical stock prices
tickers = ['AAPL', 'MSFT', 'GOOGL']
df = yf.download(
    tickers,
    interval="1d",
    period="max",
    progress=False,
).Close.bfill()
df

Ticker,AAPL,GOOGL,MSFT
Date,,,
1980-12-12,0.128348,2.511011,0.097222
1980-12-15,0.121652,2.511011,0.097222
1980-12-16,0.112723,2.511011,0.097222
1980-12-17,0.115513,2.511011,0.097222
1980-12-18,0.118862,2.511011,0.097222
...,...,...,...
2024-11-04,222.009995,169.240005,408.459991
2024-11-05,223.449997,169.740005,411.459991
2024-11-06,222.720001,176.509995,420.179993


In [96]:

def plot_asset_prices(period, shift):
    end_date = df.index[-1] - timedelta(days=shift)
    start_date = end_date - timedelta(days=period)
    df_normalized = df[(df.index >= start_date) & (df.index <= end_date)]
    df_normalized = df_normalized / df_normalized.iloc[0] - 1
    df_normalized.reset_index(names='Date', inplace=True)
    return gr.LinePlot(
        value=pd.melt(df_normalized, id_vars=['Date'], var_name='Asset', value_name='Price'),
        x='Date',
        y='Price',
        color='Asset',
        title="Normalized Asset Prices",
        y_title="Relative Change",
        x_label_angle=45,
        # height=600,
    )


with gr.Blocks() as demo:
    plot = plot_asset_prices(365, 0)
    with gr.Row():
        period = gr.Radio(
            choices=[('5y', 5*365), ('3y', 2*365), ('2y', 2*365), ('1y', 365), ('6mo', 182), ('1mo', 30), ('1w', 7)],
            value=365,
            label="Period",
        )
        shift = gr.Slider(minimum=0, maximum=365, value=0, label="End Date")
    period.change(plot_asset_prices, inputs=[period, shift], outputs=plot)
    shift.change(plot_asset_prices, inputs=[period, shift], outputs=plot)

demo.launch()


* Running on local URL:  http://127.0.0.1:7905

To create a public link, set `share=True` in `launch()`.


In [78]:
import gradio as gr
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Sample data generation
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', periods=100, freq='D')
stocks = ['Stock A', 'Stock B', 'Stock C']
data = {stock: np.random.normal(loc=100, scale=10, size=len(dates)) for stock in stocks}
df = pd.DataFrame(data, index=dates)

# Normalize prices at t=0
df_normalized = df / df.iloc[0] * 100
df_normalized.reset_index(inplace=True)
df_normalized = pd.melt(df_normalized, id_vars=['index'], var_name='Stock', value_name='Price')
df_normalized.rename(columns={'index': 'Date'}, inplace=True)

# Function to create the plot
def create_plot():
    fig = go.Figure()
    for stock in df_normalized['Stock'].unique():
        stock_data = df_normalized[df_normalized['Stock'] == stock]
        fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Price'], mode='lines', name=stock))
    fig.update_layout(title='Normalized Stock Prices', xaxis_title='Date', yaxis_title='Price',
                      xaxis_rangeslider_visible=True)
    return fig

# Gradio interface
with gr.Blocks() as demo:
    plot = gr.Plot(create_plot)
    gr.Markdown("Use the range slider below the plot to adjust the time frame.")
    gr.Markdown("You can also pan and zoom directly within the plot.")

demo.launch()


* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.
